In [ ]:
%pip install requests pyjwt openai openpyxl python-dotenv pandas

# 安装依赖包

In [ ]:
import jwt
import time
import requests
import openai
import pandas as pd
import openpyxl
import os
from dotenv import load_dotenv
import json
from json.decoder import JSONDecodeError

In [ ]:
# 获取.env 文件中的 配置
load_dotenv()

KEY_ID = os.getenv('KEY_ID')
ISSUER_ID = os.getenv('ISSUER_ID')
KEY_FILE = os.getenv('KEY_FILE')
BUNDLE_ID = os.getenv('BUNDLE_ID')
openai.api_key=os.getenv("OPENAI_API_KEY")

In [ ]:
# 获取 appconnect 的授权token
def generate_token():
    # 载入 private key
    with open(KEY_FILE, "r") as f:
        private_key = f.read()

    headers = {
        "alg": "ES256",
        "kid": KEY_ID,
        "typ": "JWT"
    }
    payload = {
        "iss": ISSUER_ID,
        "exp": time.time() + 1200,  # token 20 分钟过期
        "aud": "appstoreconnect-v1"
    }

    # 生成 token
    token = jwt.encode(payload, private_key, algorithm="ES256", headers=headers)
    return token

In [ ]:
token = generate_token()
print(token)

In [ ]:
# 设置通用的 headers
headers = {
        "Authorization": f"Bearer {token}"
    }

In [ ]:
# 获取 app_id
def get_app_id():

    headers = {
        "Authorization": f"Bearer {token}"
}

    params = {
        "filter[bundleId]": BUNDLE_ID
    }
    response = requests.get("https://api.appstoreconnect.apple.com/v1/apps", headers=headers, params=params)
    response_json = response.json()
    
    app_id  = response_json['data'][0]['id'] if response_json['data'] else None
    return app_id

APP_ID = get_app_id()
print(APP_ID)

In [ ]:
# 获取 appinfos 信息，并通过里面的 appStoreState 来获取未提交的 App 版本的 id
def get_app_info(app_id):
    url = f"https://api.appstoreconnect.apple.com/v1/apps/{app_id}/appInfos"
    response = requests.get(url=url, headers=headers)
    app_info_data = response.json()
    return app_info_data

data = get_app_info(APP_ID)
app_info_id_not_sale = [item['id'] for item in data['data'] if item['attributes']['appStoreState'] != 'READY_FOR_SALE'].pop()
print(app_info_id_not_sale)


In [ ]:
# 通过上面获取的 app_info_id 来获取该版本下的所有国际化的内容
def get_localizations(app_info_id):
    url = f"https://api.appstoreconnect.apple.com/v1/appInfos/{app_info_id}/appInfoLocalizations"
    response = requests.get(url=url, headers=headers)
    localizations_data = response.json()
    return localizations_data

localizations = get_localizations( app_info_id_not_sale)
print(localizations)

In [ ]:
# 将 localizations 组装成 dataframe 方便处理
records = [
    {
        'id': item['id'],
        'locale': item['attributes']['locale'],
        'name': item['attributes']['name'],
        'subtitle': item['attributes']['subtitle'],
        'link': item['links']['self'],
    }
    for item in localizations['data']
]

localesData = pd.DataFrame(records)

In [ ]:
localesData.head()

In [ ]:
localesData.columns

In [ ]:
# 保存 localesData 到本地 Excel 中
localesData.to_excel("store.xlsx")

In [ ]:
# 使用 loc 方法找到 'locale' 列中值为 'id' 的行，并将这些行的 'locale' 值设置为 'id-id',方便 gpt  来处理
localesData.loc[localesData['locale'] == 'id', 'locale'] = 'id-id'

In [ ]:
# 这里是过滤掉以 zh 开头的行，不处理中文内容，可选
data = localesData[~localesData['locale'].str.startswith('zh')]

In [ ]:
# 获取 gpt 的 model response
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

In [ ]:
# GPT 的prompt 定义

def transcribe_by_identifier(text,identifier):
    prompt = f"""
    将``` ``` 中的内容翻译成语言地区标识符为"{identifier}"的语言
    ```{text}```
    输出结果结构如下,translations 中如遇到换行则必须用 \\n 代替。一定不要出现符号':'，如果出现':'则用 ','代替 
    {{
    "language":"english",
    "identifier":"en-us",
    "translations": "stay \\n hungry"
    }}
    """
    response = get_completion(prompt)
    #print(response)
    # time.sleep(3) 如果 gpt 反馈调用太频繁，这里可以这么设置
    return response

In [ ]:
# 处理 App name 每一行，增加翻译内容
text = f'''
    这里是你要翻译处理的 App name
    '''

def trans_row_name(row):
    identifier = row['locale']
    print(identifier)
    transcribe_response = transcribe_by_identifier(text=text,identifier=identifier)
    transcribed = json.loads(transcribe_response)
    row['language']=transcribed["language"]
    row['new_identifier']=transcribed["identifier"]
    row['new_name']= transcribed["translations"]

    return row

In [ ]:
# 对每一行使用上面的翻译函数，增加翻译结果到 new_name 列
new_data = data.apply(trans_row_name,axis=1)
new_data

In [ ]:
text = f"""
    输入 App 的副标题
    """

# 定义一个处理每一行的函数
def trans_row_subtitle(row):
    identifier = row['locale']
    print(identifier)
    transcribe_response = transcribe_by_identifier(text=text,identifier=identifier)
    transcribed = json.loads(transcribe_response)
    row['new_subtitle']= transcribed["translations"]

    return row

In [ ]:
# 给数据加上 副标题 的翻译结果
new_data = new_data.apply(trans_row_subtitle,axis=1)

In [ ]:
new_data.head()

In [ ]:
# 保存到 Excel ，可以方便检查翻译结果
new_data.to_excel("new_name.xlsx")

In [ ]:
# 载入待更新的 data 表格，其中 new_name 为翻译的App name ，new_subtitle 为副标题
send_data = pd.read_excel("new_name.xlsx",index_col=0)
send_data

In [ ]:
# 执行更新 App name
def update_name(new_name,id) :
    url = f"https://api.appstoreconnect.apple.com/v1/appInfoLocalizations/{id}"
    
    payload = {
        'data': {
            'type': 'appInfoLocalizations',
            'id': id,
            'attributes': {
                'name': new_name  # 想要替换的名字
            }
        }
    }

    response = requests.patch(url=url, headers=headers, json=payload)

    if response.ok:
        return True
    else:
        return False

In [ ]:
# 执行更新 subtitle的函数
def update_subtitle(new_subtitle,id) :
    url = f"https://api.appstoreconnect.apple.com/v1/appInfoLocalizations/{id}"
    
    payload = {
        'data': {
            'type': 'appInfoLocalizations',
            'id': id,
            'attributes': {
                'subtitle': new_subtitle  # 想要替换的名字
            }
        }
    }

    response = requests.patch(url=url, headers=headers, json=payload)

    if response.ok:
        return True
    else:
        return False

In [ ]:
# 执行更新 name 的函数
def send_row_name(row):
    result = update_name(new_name=row['new_name'],id=row['id'])
    row['result']=result
    print(f"更新了{row['language']}:{row['new_name']} - { '成功' if result else '失败' }")
    return row

In [ ]:
# 执行更新副标题的函数
def send_row_subtitle(row):
    result = update_subtitle(new_subtitle=row['new_subtitle'],id=row['id'])
    row['result_subtitle']=result
    print(f"更新了{row['language']}:{row['result_subtitle']} - { '成功' if result else '失败' }")
    return row

In [ ]:
# 对加了翻译结果的dataframe 执行 更新 name
sended_data = send_data.apply(send_row_name,axis=1)

In [ ]:
# 对加了翻译结果的 dataframe 执行更新 副标题
sended_subtitle = send_data.apply(send_row_subtitle,axis=1)

# 从下面开始是对 更新说明 whatsnew 、应用描述的操作

In [ ]:
# 获取 app_store_version_localization_id
def get_app_store_version_localization_id(app_id, headers):
    # 获取 the App Store Version
    url_versions = f"https://api.appstoreconnect.apple.com/v1/apps/{app_id}/appStoreVersions"
    response_versions = requests.get(url=url_versions, headers=headers)
    apple_versions_data = response_versions.json()
    
    # 这里需要查看下结果，默认第一个是最新的版本
    apple_version_id = apple_versions_data['data'][0]['id']

    # 获取 the App Store Version Localization
    url_localizations = f"https://api.appstoreconnect.apple.com/v1/appStoreVersions/{apple_version_id}/appStoreVersionLocalizations"
    response_localizations = requests.get(url=url_localizations, headers=headers)
    localizations_data = response_localizations.json()

    # 获取每个 locale 下的 id
    rows=[]

    # 遍历 data 字段中的每个条目
    for item in localizations_data['data']:
        # 提取所需的字段
        row = {
            'id': item['id'],
            'locale': item['attributes']['locale'],
            'keywords': item['attributes']['keywords']
        }
        # 将提取的数据添加到列表中
        rows.append(row)

    # 创建一个 DataFrame
    localizations_whatsnew_dataframe = pd.DataFrame(rows)
  
    return localizations_whatsnew_dataframe


localizations_whatsnew_dataframe = get_app_store_version_localization_id(app_id=APP_ID,headers=headers)

In [ ]:
# 可选：过滤掉以 zh 开头的行，不处理中文内容
localizations_whatsnew_dataframe_nocn = localizations_whatsnew_dataframe[~localizations_whatsnew_dataframe['locale'].str.startswith('zh')]

# 必须：替换 locale 为 id 的行的 locale 为 id-id
localizations_whatsnew_dataframe_nocn.loc[localizations_whatsnew_dataframe_nocn['locale'] == 'id', 'locale'] = 'id-id'


In [ ]:
localizations_whatsnew_dataframe_nocn.head()

In [ ]:
# 可以保存数据
localizations_whatsnew_dataframe_nocn.to_excel("new_whatsnew_nocn.xlsx")

In [ ]:
text = f"""
    这里定义你的版本说明 whatsnew

    感谢首批用户的支持，经过加紧开发，本次给大家带来巨大更新。
    - 新 iPhonePro 用户有福了，新增支持快捷指令，配合 action button，可实现一键录音+同步转写
    - 新增支持灵动岛和实时活动，在灵动岛和实时活动中可实时显示转写出的文字，效果相当丝滑
    - 新增桌面小组件，可同步显示最新一条的语言笔记
    - 优化，语音笔记的标题同步转写结果
    - 优化，新增的语音笔记置顶
    - 优化，重新设计的主页，更精简清晰
    - 优化，修复了语言设置的 bug
    - 优化，修复了偶尔录音之后无法播放的问题
    - 优化，增加了是否显示播放控件的设置
    - 优化，增强了性能
    各位用户如果有其他需求未满足，可随时联系我，我这边肯定会每一份邮件都回复。
    """

# 翻译每一行的 whatsnew
def trans_row_whatsnew(row):
    identifier = row['locale']
    print(identifier)
    transcribe_response = transcribe_by_identifier(text=text,identifier=identifier)
    print(transcribe_response)
    try:
        # 尝试json 解码
        transcribed = json.loads(transcribe_response)
    except JSONDecodeError:
        print('Json 解码失败')
        row['whatsnew']= 'error'
        return row
        
    row['language']=transcribed["language"]
    row['new_identifier']=transcribed["identifier"]
    row['whatsnew']= transcribed["translations"]

    return row

In [ ]:
text = f"""

    这里定义你的应用介绍
    
    """

# 定义一个翻译每一行的函数
def trans_row_description(row):
    identifier = row['locale']
    print(identifier)
    transcribe_response = transcribe_by_identifier(text=text,identifier=identifier)
    print(transcribe_response)
    try:
        # 尝试json 解码
        transcribed = json.loads(transcribe_response)
    except JSONDecodeError:
        print('Json 解码失败')
        row['description']= 'error'
        return row
        
    #row['language']=transcribed["language"]
    #row['new_identifier']=transcribed["identifier"]
    row['description']= transcribed["translations"]

    return row

In [ ]:
localizations_whatsnew_dataframe_nocn_nono = pd.read_excel("new_whatsnew_nocn.xlsx",index_col=0)

In [ ]:
# 对每一行执行翻译 whatsnew
dataAddedWhatsnew = localizations_whatsnew_dataframe_nocn_nono.apply(trans_row_whatsnew,axis=1)

In [ ]:
# 对每一行执行翻译 应用描述
dataAddedDescription = localizations_whatsnew_dataframe_nocn.apply(trans_row_description,axis=1)

In [ ]:
dataAddedWhatsnew.to_excel('added_whatsnew.xlsx')

这个先保存再载入是为了人工核对翻译结果

In [ ]:
wait_send_whatsnew_data = pd.read_excel('added_whatsnew.xlsx',index_col=0)

In [ ]:
wait_send_whatsnew_data.head()

In [ ]:
# 执行更新 版本更新说明
def update_whatsnew(whatsnew,id) :
    url = f"https://api.appstoreconnect.apple.com/v1/appStoreVersionLocalizations/{id}"
    
    payload = {
        'data': {
            'type': 'appStoreVersionLocalizations',
            'id': id,
            'attributes': {
                'whatsNew': whatsnew  # 更新的说明
            }
        }
    }

    response = requests.patch(url=url, headers=headers, json=payload)

    if response.ok:
        return True
    else:
        print(response.content)
        return False
    

In [ ]:
description_data_update = pd.read_excel('addedDescription.xlsx',index_col=0)

In [ ]:
# 执行更新 应用描述 的函数
def update_description(description,id) :
    url = f"https://api.appstoreconnect.apple.com/v1/appStoreVersionLocalizations/{id}"
    
    payload = {
        'data': {
            'type': 'appStoreVersionLocalizations',
            'id': id,
            'attributes': {
                'description': description  # 更新的说明
            }
        }
    }

    response = requests.patch(url=url, headers=headers, json=payload)

    if response.ok:
        return True
    else:
        print(response.content)
        return False

In [ ]:
# 对 dataframe 批量执行  应用描述 更新的函数
def send_row_description(row):
    result = update_description(description=row['description'],id=row['id'])
    row['result_description']=result
    print(f"更新了{row['locale']}:{row['result_description']} - { '成功' if result else '失败' }")
    return row

In [ ]:
description_data_update.head()

In [ ]:
# 实际执行 应用描述 更新
description_data_update_sended = description_data_update.apply(send_row_description,axis=1)

In [ ]:
description_data_update_sended.to_excel('description_data_update_sended.xlsx')

In [ ]:
# 批量执行 whatsnew 更新的函数
def send_row_whatsnew(row):
    result = update_whatsnew(whatsnew=row['whatsnew'],id=row['id'])
    row['result']=result
    print(f"更新了{row['language']}:  { '成功' if result else '失败' }")
    return row

In [ ]:
# 实际执行 whatsnew 更新
sended_data_whatsnew = wait_send_whatsnew_data.apply(send_row_whatsnew,axis=1)

In [ ]:
# 筛选查看执行不成功的项目
sended_subtitle[sended_subtitle['result_subtitle'] == False]